I found a bunch of data from https://www.bts.gov/ and https://www.faa.gov/airports available for download. 

My goal with these data sets is to create variables I can merge together for my own data set. 

First thing: read in and preview each file...

All the data files are .xlsx or .xls (except for the one pdf?) and can be found at: /Users/lucystorts/Desktop/BYU/Stat386/386Project/PotentialData

In [70]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt             
import seaborn as sns
import json
from datetime import datetime

In [71]:
partialpath = '/Users/lucystorts/Desktop/BYU/Stat386/386Project/PotentialData/'

In [72]:
# !pip install xlrd

# !conda install -c anaconda xlrd --yes


Notes on data: 

**rankings** 
- U.S. Airports, ranked by 2022 Systemwide* Scheduled Enplanements Passenger numbers in millions (000,000)

**OriginatingPassengers**
- U.S. Airports ranked by 2022 Originating Domestic Passengers  
- Source: Bureau of Transportation Statistics, Origin & Destination Survey
- DB1B Ticket, Based on 10 Percent Ticket Sample
- O&D numbers are not comparable to T-100 Market Enplanement numbers

**airportArrivalsOnTimeCurrentMonthly**
- Table 3
- Ranking of Major Airport On-Time Arrival Performance in for Sept 22 - Aug 23					
- Based on Marketing Carrier Performance, (Percent On-Time)					

**airportArrivalsOnTimeHistorical**
- Table 4
- Ranking of Major Airport On-Time Arrival Performance Year-to-date through December
- tables for 2004 - 2022						
- Based on Marketing Carrier Performance, (Percent On-Time)		

**airportDepartureOnTimeCurrentMonthly**
- Table 5
- Ranking of Major Airport On-Time Departure Performance for Sept 22 - Aug 23						
- Based on Marketing Carrier Performance, (Percent On-Time)								

In [73]:
rankings = pd.read_excel(partialpath+'AirportRankings2022.xlsx')
originatingPassengers = pd.read_excel(partialpath+'OriginatingPassengers.xlsx')
airportLOCID = pd.read_excel(partialpath+'NPIAS-Report-2019-2023-Appendix-A.xlsx')
airportPFC = pd.read_excel(partialpath+'pfc_airports_20231031_0.xlsx')
airportGrants = pd.read_excel(partialpath+'airportGrants.xlsx')

/Users/lucystorts/opt/anaconda3/envs/stat386/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [74]:
onTimePath = partialpath+'AnnualAirlineOn-TimeRankings2003-2020.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airlineArrivalsOnTime = {}
for sheetName in sheetNames:
    airlineArrivalsOnTime[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airlineArrivalsOnTime['2003'].head(3)

,Unnamed: 0,Carrier,On-Time Arrival Pct.
0,1,Hawaiian,90.77
1,2,SkyWest,87.59
2,3,Southwest,86.30


In [75]:
onTimePath = partialpath+'Table3-RankingMajorAirportOn-TimeArrivalPerformanceAugust2023.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportArrivalsOnTimeCurrentMonthly = {}
for sheetName in sheetNames:
    airportArrivalsOnTimeCurrentMonthly[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airportArrivalsOnTimeCurrentMonthly['Aug23'].head(3)

,Rank,2022-08-01 00:00:00,%,Rank.1,2023-08-01 00:00:00,%.1,Unnamed: 6,
0,1,"Salt Lake City, UT (SLC)",84.62,1.0,"Salt Lake City, UT (SLC)",83.87,NaN,NaN
1,2,"Seattle, WA (SEA)",84.40,2.0,"Minneapolis/St. Paul, MN (MSP)",83.74,NaN,NaN
2,3,"Minneapolis/St. Paul, MN (MSP)",83.07,3.0,"Washington, DC (IAD)",82.91,NaN,NaN


In [76]:
onTimePath = partialpath+'Table4RankingMajorAirportOn-TimeArrivalPerformanceYear-to-dateDecember2003-Dec2022.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportArrivalsOnTimeHistorical = {}
for sheetName in sheetNames:
    airportArrivalsOnTimeHistorical[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airportArrivalsOnTimeHistorical['2022'].head(3)

,Rank,"Jan 1 - December 31, 2021",%,Rank.1,"Jan 1 - December 31, 2022",%.1
0,1,"Atlanta, GA (ATL)",87.98,1.0,"Salt Lake City, UT (SLC)",84.32
1,2,"Minneapolis/St. Paul, MN (MSP)",87.52,2.0,"Atlanta, GA (ATL)",82.38
2,3,"Charlotte, NC (CLT)",86.43,3.0,"Minneapolis/St. Paul, MN (MSP)",81.94


In [122]:
onTimePath = partialpath+'Table5-RankingMajorAirportOn-TimeDeparturePerformanceAugust2023.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportDepartureOnTimeCurrentMonthly = {}
for sheetName in sheetNames:
    airportDepartureOnTimeCurrentMonthly[sheetName] = excelFile.parse(sheetName)

airportDepartureOnTimeCurrentMonthly['Aug23'].head(3)

,Rank,2022-08-01 00:00:00,%,Rank.1,2023-08-01 00:00:00,%.1
0,1,"Salt Lake City, UT (SLC)",83.78,1.0,"Salt Lake City, UT (SLC)",82.55
1,2,"San Francisco, CA (SFO)",82.89,2.0,"Minneapolis/St. Paul, MN (MSP)",81.81
2,3,"Detroit, MI (DTW)",82.86,3.0,"Seattle, WA (SEA)",81.65


In [78]:
rankings.head(3)

,2022 Rank,Airport,2022 Enplaned Passengers,2021 Rank,2021 Enplaned Passengers,Pct. Change 2021-2022
0,1,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",45.3721,1.0,36.6658,23.745016
1,2,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",35.3294,2.0,30.0006,17.762311
2,3,"Denver, CO: Denver International",33.7499,3.0,28.6285,17.889166


In [79]:
originatingPassengers.head(3)

,Unnamed: 0,Airport,Code,Originating Domestic Passengers
0,1.0,"Los Angeles, CA: Los Angeles International",LAX,1324394
1,2.0,"Denver, CO: Denver International",DEN,1138192
2,3.0,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",ATL,1088364


In [80]:
airportLOCID.head(3)

,State,City,Airport,LOCID,Ownership,Hub,Role,Current,Year 5,Enplaned,Based,2019-2023 Dev Estimate
0,AK,Adak Island,Adak,ADK,PU,NaN,Basic,GA,GA,2151,0,16166667
1,AK,Akhiok,Akhiok,AKK,PU,NaN,Basic,GA,GA,399,0,8161404
2,AK,Akiachak,Akiachak,Z13,NaN,NaN,Local,CS,GA,3045,0,0


In [81]:
airportPFC.head(3)

,Associated City,State,Airport Name,LOC ID,Hub size,Level,Start Date,Expiration Date,Total PFC Approved (by location)
0,Fairbanks,AK,Fairbanks International,FAI,S,3.0,2000-10-01,2004-04-01,NaN
1,Fairbanks,AK,Fairbanks International,FAI,S,4.5,2004-04-01,2026-10-01,38413252.0
2,Juneau,AK,Juneau International,JNU,N,3.0,1998-10-01,2001-02-01,NaN


In [83]:
airportGrants.head(3)

,State,City,Airport,LOCID,Project Description,Total Amount
0,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,Girdwood,Girdwood,AQY,Seal Apron Pavement Surface/Pavement Joints,504078.0
2,NaN,NaN,NaN,NaN,NaN,NaN


Now, I want to create datasets for 2022. 

Pandas dataframes all by themselves:
- rankings
- originatingPassengers
- airportLOCID
- airportPFC
- airportPassengersBoarded
- airportGrants 

Other objects:
- airlineArrivalsOnTime
- airportArrivalsOnTimeCurrentMonthly
- airportArrivalsOnTimeHistorical
- airportDepartureOnTimeCurrentMonthly

** need to be accessed with ['key'] to return the pandas dataframe. 

First thing, I need to use airportLOCID to create index outline to fill my new dataframe. 

I am going to create a set of clean objects that I can merge together somehow in the end. 

In [84]:
columns = ['LOCID', 'Airport','City','State']
cleanLOCID = pd.concat([airportLOCID['LOCID'], airportLOCID['Airport'],airportLOCID['City'],airportLOCID['State']],axis=1)
cleanLOCID.head(10)

,LOCID,Airport,City,State
0,ADK,Adak,Adak Island,AK
1,AKK,Akhiok,Akhiok,AK
2,Z13,Akiachak,Akiachak,AK
3,AKI,Akiak,Akiak,AK
4,7AK,Akutan,Akutan,AK
5,KQA,Akutan,Akutan,AK
6,AUK,Alakanuk,Alakanuk,AK
7,5A8,Aleknagik/New,Aleknagik,AK
8,6A8,Allakaket,Allakaket,AK
9,AFM,Ambler,Ambler,AK


Now, specific to 2022, I am going to pull data from rankings, originatingPassengers, airportPassengersBoarded, and possibly airportGrants. 

In [85]:
# clean for bottom headers 
rankings = rankings.head(200)

# Use this to find LOCID for ranked airports 
cleanLOCID[(cleanLOCID['Airport'] == 'Hartsfield-Jackson Atlanta International')]['LOCID']

rankedAirports = rankings['Airport']
rankedAirportNames = rankedAirports.str.extract(r':\s*(.*)$')

rankedLOCID = []
count = 0

for i in range(len(rankedAirportNames)):
    try:
        rankedLOCID += [cleanLOCID.loc[cleanLOCID['Airport'] == rankedAirportNames[0][i] , 'LOCID'].iloc[0]]
    except IndexError:
        rankedLOCID += [rankedAirportNames[0][i]]
        count += 1 # This lets me know how many didn't match up from my cleanLOCID DF

rank = rankings['2022 Rank']
conversionfactor = 1000000
numPassengers = rankings['2022 Enplaned Passengers']*conversionfactor


I  need to clean the rankedLOCID one more time using a different LOCID reference. 

I am going to request the LOCID values for the unknown airports from chatGPT in a .csv file. 

In [86]:
# Use for chatGPT request
for i in range(len(rankedLOCID)):
    if len(rankedLOCID[i]) > 3:
        #print(rankedLOCID[i]) 
        next

extraLOCID = pd.read_csv(partialpath+'extraLOCID.txt')
extraLOCID.head(3)


,Airport,LOCID
0,Dallas/Fort Worth International,DFW
1,John F. Kennedy International,JFK
2,Harry Reid International,LAS


In [87]:
cleanedRankedLOCID = []

for i in range(len(rankedLOCID)):
    if len(rankedLOCID[i]) > 3:
        cleanedRankedLOCID += [extraLOCID.loc[extraLOCID['Airport'] == rankedLOCID[i] , 'LOCID'].iloc[0]]
    else:
        cleanedRankedLOCID += [rankedLOCID[i]]

In [88]:
# Combine together for initial dataframe: 

cleanRank = pd.concat([rank,pd.DataFrame(cleanedRankedLOCID),numPassengers],axis=1)
cleanRank.rename(columns={0: 'Airport'}, inplace=True)
cleanRank.head(10)

,2022 Rank,Airport,2022 Enplaned Passengers
0,1,ATL,45372100.0
1,2,DFW,35329400.0
2,3,DEN,33749900.0
3,4,ORD,33100400.0
4,5,LAX,32271400.0
5,6,JFK,26923300.0
6,7,LAS,25264700.0
7,8,MCO,24431300.0
8,9,MIA,23673000.0
9,10,CLT,23085500.0


Now, I want to keep adding more information to this table. 

I am going to work with originatingPassengers. 

originatingPassengers has a column for Code to match up to Airport, then add new column, Originating Domestic Passengers. 

In [89]:
cleanCodes = cleanRank['Airport']
ogDomPassengers = []
originatingPassengers.columns = originatingPassengers.columns.str.strip()

for i in range(len(cleanCodes)):
    try:
        ogDomPassengers += [originatingPassengers.loc[originatingPassengers['Code'] == cleanCodes[i] , 'Originating Domestic Passengers'].iloc[0]]
    except IndexError:
        ogDomPassengers += [None]


Add this new variable to the existing table... 

In [91]:
cleanedData = pd.concat([cleanRank, pd.DataFrame(ogDomPassengers)],axis=1)
cleanedData.rename(columns={0: '2022 Originating Domestic Passengers'}, inplace=True)
cleanedData.head(5)

,2022 Rank,Airport,2022 Enplaned Passengers,2022 Originating Domestic Passengers
0,1,ATL,45372100.0,1088364.0
1,2,DFW,35329400.0,859264.0
2,3,DEN,33749900.0,1138192.0
3,4,ORD,33100400.0,1085583.0
4,5,LAX,32271400.0,1324394.0


Next, I want to add grants information. 

airportGrants needs to be cleaned for null rows and the columns need to be stripped of white space. 

In [118]:
cleanGrants = airportGrants.dropna().reset_index(drop=True)
cleanGrants.columns = cleanGrants.columns.str.strip()
cleanGrants['LOCID'] = cleanGrants['LOCID'].str.replace('*', '', regex=True)
#cleanGrants['Total Amount'].groupby(cleanGrants['LOCID'])
groupedGrants = cleanGrants.groupby('LOCID')['Total Amount'].sum()
groupedGrants.keys()

Index(['00U ', '2R5 ', '9S5 ', 'AQY ', 'ATL ', 'BFI ', 'CVG ', 'CWA ', 'CYS ',
       'DCA ', 'EAT ', 'EHR ', 'GFK ', 'GRK ', 'GXY ', 'HRL ', 'HSV ', 'HUM ',
       'HVN ', 'HYA ', 'ILB ', 'ILS ', 'KSM ', 'L83 ', 'LAX ', 'LFT ', 'MIB ',
       'MKC ', 'MRY ', 'MSL ', 'MSO ', 'N19 ', 'PGD ', 'SAN ', 'SBN ', 'SDY ',
       'SFO ', 'SLN ', 'TCL ', 'TXB '],
      dtype='object', name='LOCID')

Compare how many airports in cleanedData will be matched up with airports given grant money to see if it's worth it to have 150+ null vals 

In [125]:
for i in range(len(groupedGrants)):
    #groupedGrants.keys()[i]
    next

#cleanedData['Airport'] == groupedGrants.keys()

Eventually, I want to use the API to request longitude and lattitude for each airport in this ranked list. 